In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
base_blend_with_ambient = False

In [3]:
# code from https://stackoverflow.com/a/43346070
def gkern(l, sig = 1.):
    """\
    creates gaussian kernel with side length `l` and a sigma of `sig`
    """
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

In [4]:
def flash_no_flash_process(k, a, f, d):
    # Load images
    A = cv2.imread('./Test_Photos/'+d+'/'+a)
    F = cv2.imread('./Test_Photos/'+d+'/'+f)
    
    if A.shape[0] * A.shape[1]/1000000 < 1:
        sigmaSpace = 3
    elif A.shape[0] * A.shape[1]/1000000 < 2:
        sigmaSpace = 9
    else:
        sigmaSpace = 36
        
    sigmaColor = round(255 * 0.1)
    
    # Bilateral filter on ambient image
    A_base = cv2.bilateralFilter(A,sigmaSpace,sigmaColor,sigmaSpace)
    
    # Joint bilateral filter with flash image as the guide
    A_nr = cv2.ximgproc.jointBilateralFilter(F, A, sigmaSpace,round(255 * 0.001),sigmaSpace)
    
    # Extract detail of flash image
    epsilon = np.ones(F.shape, dtype = np.float32) * 0.02
    F_base = cv2.bilateralFilter(F,sigmaSpace, 50,sigmaSpace)
    F_detail = cv2.divide(F.astype(np.float32) + epsilon, F_base.astype(np.float32) + epsilon)
    F_detail = np.clip(F_detail,0,2)
    A_nrdetail = cv2.multiply(F_detail, A_nr.astype(np.float32))
    A_nrdetail = np.clip(A_nrdetail, 0, 255).astype(np.uint8)
    cv2.imwrite('./Output/'+d+'/'+k+'_ambient_nrdetail.png', A_nrdetail)
        
    # Heuristic of shadow and specularity detection
    A_grey = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)
    A_grey = cv2.GaussianBlur(A_grey,(5,5),0)
    A_grey = cv2.normalize(A_grey, None, 0, 255, cv2.NORM_MINMAX)
    F_grey = cv2.cvtColor(F, cv2.COLOR_BGR2GRAY)
    F_grey = cv2.GaussianBlur(F_grey,(5,5),0)
    F_grey = cv2.normalize(F_grey, None, 0, 255, cv2.NORM_MINMAX)
    
    A_hist = cv2.calcHist(A_grey,[0],None,[256],[0,256])
    A_hist = np.cumsum(A_hist)
    A_shadIndex = np.searchsorted(A_hist, round(A_hist[-1]*0.05), side='left')

    F_hist = cv2.calcHist(F_grey,[0],None,[256],[0,256])
    F_hist = np.cumsum(F_hist)
    F_shadIndex = np.searchsorted(F_hist, round(F_hist[-1]*0.2), side='left')
    F_specIndex = 245
    

    _, F_shadow = cv2.threshold(F_grey, F_shadIndex, 255, cv2.THRESH_BINARY_INV)
    _, A_shadow = cv2.threshold(A_grey, A_shadIndex, 255, cv2.THRESH_BINARY_INV)
    
    kernel = gkern(5)
    F_shadow = cv2.dilate(F_shadow,kernel,iterations = 1)
    A_shadow = cv2.erode(A_shadow,kernel,iterations = 1)
    
    M_shadow = cv2.bitwise_and(F_shadow, cv2.bitwise_not(A_shadow))
    M_shadow = M_shadow.astype(np.uint8)
    M_shadow = cv2.dilate(M_shadow,kernel,iterations = 1)
    
    _, F_specular = cv2.threshold(F_grey, F_specIndex, 255, cv2.THRESH_BINARY)
    F_specular = cv2.dilate(F_specular,kernel,iterations = 1)
    
    M_art = cv2.bitwise_or(M_shadow, F_specular)
    cv2.imwrite('./Output/'+d+'/'+k+'_mask_artifact.png', M_art)
        
    # Save output image
    A_final = np.zeros(A.shape, dtype = np.uint8)
    for i in range(3):
        A_final[:,:,i] = cv2.bitwise_or(cv2.bitwise_and(A_nrdetail[:,:,i],cv2.bitwise_not(M_art)), cv2.bitwise_and(A_base[:,:,i],M_art))
    cv2.imwrite('./Output/'+d+'/'+k+'_ambient_final.png',  A_final)
    
    if base_blend_with_ambient:
        A_final = np.zeros(A.shape, dtype = np.uint8)
        A_blend = cv2.addWeighted(A, 0.5, A_base, 0.5, 0.0)
        for i in range(3):
            A_final[:,:,i] = cv2.bitwise_or(cv2.bitwise_and(A_nrdetail[:,:,i],cv2.bitwise_not(M_art)), cv2.bitwise_and(A_base[:,:,i],M_art))
        cv2.imwrite('./Output/'+d+'/'+k+'_ambient_final_blend.png',  A_final)

In [5]:
import os
import csv
if not os.path.exists('./Output'):
    os.mkdir('./Output')
directories = os.listdir('Test_Photos')
for d in directories:
    print(d)
    if not os.path.exists('./Output/'+d):
        os.mkdir('./Output/'+d)
    with open('./Test_Photos/'+d+'/meta.csv', 'r') as csvfile:
        r = csv.DictReader(csvfile)
        for row in r:
            flash_no_flash_process(row['key'], row['ambient'], row['flash'], d)

flash_data_JBF_Detail_transfer
Object_Photos
people
plants
Rooms_Photos
shelves
Toys_Photos
